# Federated Learning via Trusted Aggregator

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
import syft as sy
import copy
hook = sy.TorchHook(torch)

W0710 13:08:23.466204 140393701947200 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/home/sijoonlee/anaconda3/envs/pysyft/lib/python3.7/site-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0710 13:08:23.486423 140393701947200 deprecation_wrapper.py:119] From /home/sijoonlee/anaconda3/envs/pysyft/lib/python3.7/site-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [3]:
# create a couple workers
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
secure_worker = sy.VirtualWorker(hook, id="secure_worker")

In [4]:
# transforms
train_transforms = transforms.Compose([#transforms.RandomRotation(30),
                                       # transforms.RandomResizedCrop(224),
                                       # transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.5,], [0.5,])]) # mean, std
 

test_transforms = transforms.Compose([#transforms.Resize(255),
                                      #transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.5,], [0.5,])]) # mean, std


# choose the training and test datasets

federated_train_loader = sy.FederatedDataLoader( # <-- this is now a FederatedDataLoader 
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))
    .federate((bob, alice)), # <-- NEW: we distribute the dataset across all the workers, it's now a FederatedDataset
    batch_size=20, shuffle=True)

federated_test_loader = sy.FederatedDataLoader( # <-- this is now a FederatedDataLoader 
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))
    .federate((bob, alice)), # <-- NEW: we distribute the dataset across all the workers, it's now a FederatedDataset
    batch_size=20, shuffle=True)



In [5]:
# distribute train_data to Bob and Alice

import numpy as np

from torch.utils.data import TensorDataset

# sampleFromClass source code from
# https://stackoverflow.com/questions/50544730/how-do-i-split-a-custom-dataset-into-training-and-test-datasets
def sampleFromClass(ds, k):
    class_counts = {}
    one_data = []
    one_label = []
    another_data = []
    another_label = []
    for data, label in ds:
        # c = label.item()
        c = label
        class_counts[c] = class_counts.get(c, 0) + 1
        if class_counts[c] <= k:
            one_data.append(data)
            one_label.append(label)
        else:
            another_data.append(data)
            another_label.append(label)
            
    one_data = torch.cat(one_data)
    one_label = torch.IntTensor(one_label)
    another_data = torch.cat(another_data)
    another_label = torch.IntTensor(another_label)
    #another_label = torch.cat(another_label)

    return one_data, one_label, another_data, another_label


bob_data, bob_label, alice_data, alice_label = sampleFromClass(train_data, 3000)
bob_data = bob_data.send(bob)
bob_label = bob_label.send(bob)
alice_data = alice_data.send(alice)
alice_label = alice_label.send(alice)

print(bob_data)

(Wrapper)>[PointerTensor | me:45126947390 -> bob:92650601097]


In [7]:
test_loader = torch.utils.data.DataLoader(test_data, batch_size=20)
test_loader.send(bob)

AttributeError: 'DataLoader' object has no attribute 'send'

In [21]:
# define the CNN architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
       # convolutional layer (sees 28x28x1 image tensor)
        self.conv1 = nn.Conv2d(1, 4, 3, padding=1)
        # convolutional layer (sees 14x14x4 tensor after MaxPool)
        self.conv2 = nn.Conv2d(4, 16, 3, padding=1)
        # max pooling layer
        self.pool = nn.MaxPool2d(2, 2)
        # linear layer (7 * 7 * 16)
        self.fc1 = nn.Linear(7 * 7 * 16, 512)
        # linear layer (512 -> 10)
        self.fc2 = nn.Linear(512, 10)
        # dropout layer (p=0.20)
        self.dropout = nn.Dropout(0.20)
        
        
    def forward(self, x):
        # add sequence of convolutional and max pooling layers
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        # flatten image input
        x = x.view(-1, 7 * 7 * 16)
        # add dropout layer
        x = self.dropout(x)
        # add 1st hidden layer, with relu activation function
        x = F.relu(self.fc1(x))
        # add dropout layer
        x = self.dropout(x)
        # add 2nd hidden layer, with relu activation function
        x = self.fc2(x)
        # LogSoftMax
        return F.log_softmax(x, dim=1)

In [22]:
model = Net()

bob_model = model.copy().send(bob)
bob_optimizer = optim.Adam(model.parameters(), lr=0.003)

alice_model = model.copy().send(alice)
alice_optimizer = optim.Adam(model.parameters(), lr=0.003)

criterion = nn.NLLLoss()

In [23]:
def train(model, criterion, optimizer, data, label, epoches, batch_size=20):

    train_on_gpu = torch.cuda.is_available()

    if train_on_gpu:
        print('CUDA is available!  Training on GPU ...')
        model.cuda()

    train_losses, test_losses = [], []
    
    train_data = TensorDataset(data, label)
    train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size)
    
    for epoch in range(epoches):

        train_loss, test_loss = 0.0, 0.0

        model.train()

        for images, labels in train_loader:

            if train_on_gpu:
                images, labels = images.cuda(), labels.cuda()

            optimizer.zero_grad()

            log_ps = model(images)
            loss = criterion(log_ps, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        model.eval()
        with torch.no_grad():
            for images, labels in test_loader:
                # move tensors to GPU if CUDA is available
                if train_on_gpu:
                    images, labels = images.cuda(), labels.cuda()

                # forward pass: compute predicted outputs by passing inputs to the model
                output = model(images)
                # calculate the batch loss
                loss = criterion(output, labels)
                # update average validation loss 
                test_loss += loss.item()

        # calculate average losses
        train_loss = train_loss/len(train_loader)
        test_loss = test_loss/len(test_loader)

        train_losses.append(train_loss)
        test_losses.append(test_loss)

        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tTest Loss: {:.6f}'.format(
            epoch, train_loss, test_loss))
        
    return model, train_losses, test_losses

In [26]:
bob_model, bob_train_losses, bob_test_losses = train(bob_model, criterion, bob_optimizer, bob_data, bob_label, 5)
#alice_model, alice_train_losses, alice_test_losses = train(alice_model, criterion, alice_optimizer, alice_loader, test_loader, 5)

RuntimeError: weight should have at least three dimensions